<a href="https://colab.research.google.com/github/Nawaf9997/T5_SDAIA_Traffic_Anlysis/blob/main/CBOW_Practice_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continuous Bag of Words (CBOW) Model

In this exercise, we will create a CBOW model using a sample Arabic medical corpus. The corpus consists of sentences describing various medical scenarios. The goal of the CBOW model is to predict a target word based on its surrounding context words.

# Importing Required Libraries

In this step, we import the necessary libraries to build and train the Continuous Bag of Words (CBOW) model.

- **TensorFlow and Keras**: Used to build the neural network model, including the layers like `Embedding`, `Dense`, and `Lambda`.
- **Tokenizer**: A utility from Keras for tokenizing and processing text data.
- **NumPy**: Used for handling numerical operations, particularly for processing arrays and data manipulation.

These libraries will provide the essential tools for text preprocessing and model development in the upcoming steps.

Add more if needed!


In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Preparing the Corpus

In this step, we initialize the corpus that will be used for training the Continuous Bag of Words (CBOW) model. The corpus consists of Arabic sentences, each of which describes different medical scenarios.

- **Corpus**: A collection of medical-related sentences in Arabic, focusing on various health conditions, treatments, and patient experiences.

This step sets up the text data that we will use in the upcoming stages of tokenization and model training.

In [4]:
corpus = [
    "المريض يعاني من ارتفاع في ضغط الدم منذ فترة طويلة ويحتاج إلى تغيير نمط حياته لتجنب المضاعفات",
    "الطبيب أشار إلى ضرورة إجراء فحوصات شاملة للتأكد من عدم وجود مشاكل أخرى تؤثر على القلب والجهاز التنفسي",
    "كانت نتائج التحاليل المخبرية تشير إلى انخفاض حاد في نسبة الحديد مما يستدعي البدء في العلاج فوراً",
    "المستشفى شهد ازدحامًا شديدًا اليوم بسبب تزايد حالات الطوارئ والحالات الحرجة التي وصلت خلال ساعات الصباح",
    "المريض يحتاج إلى متابعة دورية من أجل التحكم في نسبة السكر في الدم وتجنب أي تدهور في حالته الصحية",
    "الأشعة أكدت وجود كسر في العظام ويجب على المريض الراحة التامة والالتزام بالعلاج الموصوف من الطبيب",
    "الجراحة كانت ناجحة والمريض بدأ يشعر بتحسن تدريجي في حالته ولكنه يحتاج إلى فترة نقاهة طويلة قبل العودة إلى نشاطاته",
    "الطبيب نصح المريض باتباع نظام غذائي صحي وممارسة الرياضة بانتظام للحفاظ على صحته العامة والوقاية من الأمراض",
    "تم تشخيص المريضة بمرض مزمن يتطلب علاجًا طويل الأمد وتغييرًا جذريًا في نمط الحياة للتعايش معه",
    "المريض يشعر بألم حاد في الصدر منذ عدة أيام وقد أُحيل إلى قسم القلب لإجراء فحوصات تشخيصية شاملة",
    "التحليل المخبري أكد وجود التهاب في المسالك البولية ويتطلب العلاج الفوري بالمضادات الحيوية لتفادي أي مضاعفات",
    "المريض يعاني من أعراض الأنفلونزا الموسمية ولكن حالته مستقرة ولا تحتاج إلى دخول المستشفى",
    "التهاب المفاصل تسبب في صعوبة الحركة لدى المريضة ويجب عليها البدء في العلاج الطبيعي لتخفيف الألم واستعادة الحركة",
    "الفحص الطبي الدوري كشف عن ارتفاع في نسبة الكوليسترول مما يستدعي تغييرًا في النظام الغذائي والبدء في العلاج",
    "المريضة تعاني من اضطرابات النوم بسبب القلق المفرط وقد تمت إحالتها إلى أخصائي نفسي لتقديم الدعم اللازم",
    "تم اكتشاف ورم في مرحلة مبكرة لدى المريض ويجب إجراء المزيد من الفحوصات لتحديد أفضل خيارات العلاج المتاحة",
    "التقرير الطبي أشار إلى أن المريض يعاني من مشاكل في الجهاز الهضمي ويحتاج إلى اتباع نظام غذائي خاص لتجنب الألم",
    "العملية الجراحية التي أجريت كانت ناجحة ولكن المريض يحتاج إلى مراقبة مستمرة للتأكد من عدم حدوث أي مضاعفات",
    "المريض يعاني من ضيق في التنفس وقد تم نقله إلى وحدة العناية المركزة لتلقي العلاج اللازم بشكل عاجل",
    "العلاج الطبيعي الذي يخضع له المريض يساعد في تحسين حالته بشكل ملحوظ ولكنه يحتاج إلى الاستمرار لفترة طويلة"
]

# Defining Vocabulary and Model Parameters

In this step, we define key parameters that will be used to configure the CBOW model.

- **Vocabulary size**: We calculate the size of the vocabulary based on the number of unique words in the corpus. The `vocab_size` represents the total number of unique tokens (words) in the dataset plus one for padding.
  
- **Embedding size**: The `embedding_size` defines the dimensionality of the word embeddings. In this case, we set the embedding size to 10, meaning each word will be represented as a 10-dimensional vector in the embedding layer.

- **Window size**: The `window_size` defines how many words to the left and right of the target word are considered as context. Here, a window size of 2 means that two words before and two words after the target word will be used as context.

These parameters will play an essential role in shaping the CBOW model architecture.


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
print('After converting our words in the corpus into vector of integers:')
print(sequences)

After converting our words in the corpus into vector of integers:
[[3, 6, 4, 20, 1, 57, 21, 22, 23, 9, 24, 2, 58, 25, 59, 26, 60], [10, 27, 2, 61, 28, 29, 30, 31, 4, 32, 11, 33, 62, 63, 12, 34, 64, 65], [13, 66, 67, 68, 69, 2, 70, 35, 1, 14, 71, 36, 37, 38, 1, 5, 72], [39, 73, 74, 75, 76, 40, 77, 78, 79, 80, 81, 41, 82, 83, 84, 85], [3, 7, 2, 86, 87, 4, 88, 89, 1, 14, 90, 1, 21, 91, 15, 92, 1, 8, 93], [94, 95, 11, 96, 1, 97, 16, 12, 3, 98, 99, 100, 101, 102, 4, 10], [103, 13, 42, 104, 105, 43, 106, 107, 1, 8, 44, 7, 2, 23, 108, 9, 109, 110, 2, 111], [10, 112, 3, 113, 45, 46, 114, 115, 116, 117, 118, 12, 119, 120, 121, 4, 122], [17, 123, 18, 124, 125, 126, 127, 128, 129, 130, 131, 1, 25, 132, 133, 134], [3, 43, 135, 35, 1, 136, 22, 137, 138, 19, 139, 2, 140, 34, 141, 29, 142, 30], [143, 144, 145, 11, 47, 1, 146, 147, 148, 5, 149, 150, 151, 152, 15, 48], [3, 6, 4, 153, 154, 155, 49, 8, 156, 157, 158, 2, 159, 39], [47, 160, 161, 1, 162, 50, 51, 18, 16, 163, 38, 1, 5, 52, 164, 53, 165, 50]

In [11]:
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 10
window_size = 2

# Preparing Context-Target Pairs for CBOW

In this step, we generate the context-target pairs from the tokenized sequences to train the CBOW model.

- **Context words**: For each word in a sequence, the surrounding words (within the window size) are considered as context. The context consists of the words immediately before and after the target word.
  
- **Target word**: The word in the middle of the context window is treated as the target word that the model will learn to predict.

We iterate through each sequence, collecting the context words and corresponding target words:
- For each word in a sequence, we gather the surrounding words based on the defined window size.
- The middle word is the target, and the surrounding words form the context.

Finally:
- **`X`**: An array of context words.
- **`y`**: The target words are one-hot encoded, which means they are converted into a categorical format where each word is represented as a vector of length equal to the vocabulary size.

These context-target pairs will be used to train the CBOW model to predict a target word based on its context.


In [13]:
contexts = []
targets = []
for sequence in sequences:
    for i in range(window_size, len(sequence) - window_size):
        context = sequence[i - window_size:i] + sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)
X = np.array(contexts)
y = tf.keras.utils.to_categorical(targets, num_classes=vocab_size)

# Building and Training the CBOW Model

In this step, we build and train the Continuous Bag of Words (CBOW) model using the context-target pairs created earlier.

1. **Model architecture**:
   - **Embedding layer**: This layer transforms the input context words into dense vector representations (embeddings) of size defined by `embedding_size`. The `input_dim` is set to the vocabulary size, and the `input_length` is twice the window size (since context consists of words from both sides of the target).
   
   - **Lambda layer**: This layer computes the mean of the context word embeddings. It averages the embeddings of the context words to generate a single representation that will be used to predict the target word.
   
   - **Dense layer**: This fully connected layer outputs a probability distribution over the vocabulary, using the softmax activation function. It predicts the most likely target word based on the context word embeddings.

2. **Compilation**:
   The model is compiled using the Adam optimizer and categorical cross-entropy as the loss function, which is suitable for multi-class classification tasks. Accuracy is used as a metric to evaluate the model's performance during training.

3. **Training the model**:
   The model is trained on the context-target pairs for 500 epochs. During each epoch, the model learns to predict the target word based on the context, refining its weights to improve accuracy.

4. **Saving the model weights**:
   After training, the model weights are saved to a file (`cbow_model.weights.h5`) for future use. This allows us to load the trained model later without retraining.

By the end of this step, the CBOW model will have learned to predict target words based on their surrounding context from the given corpus.

In [28]:
embedding_size = 10
input_length = window_size * 2
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=2*window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(units=vocab_size, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=500, verbose=1)
model.save_weights('cbow_model.weights.h5')

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0042 - loss: 5.4113      
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0587 - loss: 5.4061     
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0684 - loss: 5.4018     
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1077 - loss: 5.3966 
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1266 - loss: 5.3904 
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0976 - loss: 5.3875 
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1246 - loss: 5.3818 
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1081 - loss: 5.3766 
Epoch 9/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0809 - loss: 5.3721     
Epoch 10/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1233 - loss: 5.3635 
Epoch 11/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0888 - loss: 5.3594 
Epoch 12/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accur

# Predicting a Word Using the Trained CBOW Model

In this step, we define a function to predict a word based on a given context using the trained CBOW model.

1. **Function: `predict_word`**:
   - **Input**: The function takes a list of context words as input. The number of context words should match the expected size (2 times the window size).
   - **Context sequence conversion**: The input context words are tokenized into a sequence of integers using the same tokenizer that was used during training.
   - **Input validation**: The function checks whether the length of the context sequence matches the expected size (2 times the window size). If not, it prints an error message.
   - **Prediction**: The tokenized context is fed into the trained CBOW model, which predicts the probability distribution over the vocabulary.
   - **Retrieve predicted word**: The predicted word is the one with the highest probability. The function retrieves the word corresponding to the predicted index from the tokenizer's word index.

2. **Example**:
   - We provide a sample context: `['في', 'ارتفاع', 'يعاني', 'المريض']`.
   - The function predicts the word that fits best in this context, based on the model's learned weights.
   - The predicted word is printed along with the input context.

This function allows us to test the CBOW model by predicting words based on their surrounding context from the corpus.

In [32]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer


def predict_word(context_words, tokenizer, model, window_size=2):

    context_sequence = tokenizer.texts_to_sequences([context_words])[0]


    if len(context_sequence) != window_size * 2:
        print(f"Error: The context length should be {window_size * 2}, but got {len(context_sequence)}.")
        return None


    context_sequence = np.array(context_sequence).reshape(1, -1)


    prediction = model.predict(context_sequence)


    predicted_word_index = np.argmax(prediction)


    predicted_word = None
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            predicted_word = word
            break


    print(f"Context words: {context_words}")
    print(f"Predicted word: {predicted_word}")

    return predicted_word

# Example usage
context = ['في', 'ارتفاع', 'يعاني', 'المريض']
predicted_word = predict_word(context, tokenizer, model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Context words: ['في', 'ارتفاع', 'يعاني', 'المريض']
Predicted word: من
